In [1]:
import spacy
import pandas as pd
import glob
from tqdm import tqdm
import pickle
from collections import Counter
import jsonlines
path = r'../../data/' # use your path

In [2]:
with open( "tickDict.p", "rb") as f:
    tickDict = pickle.load(f)

In [3]:
seen_tickers = set()

In [4]:
blacklist = {'I', 'ARE',  'ON', 'GO', 'NOW', 'CAN', 'UK', 'SO', 'OR', 'OUT', 'SEE', 'ONE', 'LOVE', 'U', 'STAY', 'HAS', 'BY', 'BIG', 'GOOD', 'RIDE', 'EOD', 'ELON', 'WSB', 'THE', 'A', 'ROPE', 'YOLO', 'TOS', 'CEO', 'DD', 'IT', 'OPEN', 'ATH', 'PM', 'IRS', 'FOR','DEC', 'BE', 'IMO', 'ALL', 'RH', 'EV', 'TOS', 'CFO', 'CTO', 'DD', 'BTFD', 'WSB', 'OK', 'PDT', 'RH', 'KYS', 'FD', 'TYS', 'US', 'USA', 'IT', 'ATH', 'RIP', 'BMW', 'GDP', 'OTM', 'ATM', 'ITM', 'IMO', 'LOL', 'AM', 'BE', 'PR', 'PRAY', 'PT', 'FBI', 'SEC', 'GOD', 'NOT', 'POS', 'FOMO', 'TL;DR', 'EDIT', 'STILL', 'WTF', 'RAW', 'PM', 'LMAO', 'LMFAO', 'ROFL', 'EZ', 'RED', 'BEZOS', 'TICK', 'IS', 'PM', 'LPT', 'GOAT', 'FL', 'CA', 'IL', 'MACD', 'HQ', 'OP', 'PS', 'AH', 'TL', 'JAN', 'FEB', 'JUL', 'AUG', 'SEP', 'SEPT', 'OCT', 'NOV', 'FDA', 'IV', 'ER', 'IPO', 'MILF', 'BUT', 'SSN', 'FIFA', 'USD', 'CPU', 'AT', 'GG', 'Mar'}
ticks = set(pd.read_csv(path + 'ExchangeListings/tickers.csv', index_col=None, header=0)['Symbol'])
nlp = spacy.load("en_core_web_lg")

In [5]:
all_submission_files = glob.glob(path + "*_submission.csv")

for filename in all_submission_files:
    print(filename)
    new_fn = filename[:-3] + 'jsonl'
    print(new_fn)
    new_rows = []
    df = pd.read_csv(filename, index_col=None, header=0)
    for _, row in tqdm(df.iterrows()):
        new_row = row.to_dict()
        tickCnt = Counter()
        text = f"{row['title']}\n{row['text']}"
        split = str(text).split(" ")
        for word in split:
            word = word.replace("$", "")        
            # upper = ticker, length of ticker <= 5, excluded words,                     
            if word.isupper() and len(word) <= 5 and word not in blacklist and word in ticks:
                seen_tickers.add(word)
                tickCnt[word] += 1
        doc = nlp(text)
        for e in doc.ents:
            if e.label_ == "ORG" and e.text in tickDict and e.text not in ticks:
                tick = tickDict[e.text]
                seen_tickers.add(tick)
                tickCnt[tick] += 1
        new_row['tickers'] = tickCnt.most_common()
        new_rows.append(new_row)
    with jsonlines.open(new_fn, mode='w') as writer:
        writer.write_all(new_rows)

6it [00:00, 50.42it/s]

../../data\daytrading_submission.csv
../../data\daytrading_submission.jsonl


6535it [02:12, 49.50it/s]
8it [00:00, 76.19it/s]

../../data\dividends_submission.csv
../../data\dividends_submission.jsonl


2298it [00:45, 51.00it/s]
6it [00:00, 57.69it/s]

...

3142it [01:30, 34.90it/s]


../../data\stockmarket_submission.csv
../../data\stockmarket_submission.jsonl


13136it [07:42, 28.37it/s]


../../data\stocks_submission.csv
../../data\stocks_submission.jsonl


48914it [16:44, 48.68it/s]


In [6]:
with open( "seen.p", "wb") as f:
    pickle.dump(seen_tickers, f)

In [7]:
len(seen_tickers)

4461